In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!nvidia-smi

Sat Jun  5 16:32:53 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
path = '/content/drive/Shareddrives/CLSE/SimSiam-round'

In [4]:
cd $path

/content/drive/Shareddrives/CLSE/SimSiam-round


In [5]:
# !pip install pypesq
# !pip install pystoi
!pip install einops

In [6]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import numpy as np
import random
import math
from tqdm import tqdm
# from pypesq import pesq
# from pystoi import stoi
from functools import partial
from einops.layers.torch import Rearrange, Reduce
import gc

import csv
from IPython.display import Audio

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [8]:
def get_datas(speechs, noises, snrs, length:int=16384):
  assert len(speechs) == len(noises), 'The number of voices must be equal to the number of noises'
  bz = len(speechs)
  clean_sounds = []
  noise_sounds = []
  noisy_sounds = []
  targets = []
  for i in range(bz):
    speech = np.array(speechs[i])
    if len(speech)-length<0:
      speech = np.pad(speech,(0,length-len(speech)),'wrap')
    else:
      speech_start = random.randrange(0,len(speech)-length+1)
      speech = speech[speech_start:speech_start+length]
    speech = speech - np.mean(speech) 
    speech = speech / np.linalg.norm(speech,2)
    speech_std = np.std(speech)

    noise = np.array(noises[i])
    if len(noise)-length<0:
      noise = np.pad(noise,(0,length-len(noise)),'reflect')
    else:
      noise_start = random.randrange(0,len(noise)-length+1)
      noise = noise[noise_start:noise_start+length]
    noise = noise - np.mean(noise) 
    noise = noise / np.linalg.norm(noise,2)
    noise = noise / (10.0 ** (0.05 * snrs[i]))
    noise_std = np.std(noise)

    noisy = speech + noise
    noisy_std = np.std(noisy)
    noisy = noisy / noisy_std

    speech = speech / speech_std
    
    noise = noise / noise_std
    
    clean_sounds.append(speech)
    noise_sounds.append(noise)
    noisy_sounds.append(noisy)

  clean_sounds = np.stack(clean_sounds, 0)
  clean_sounds = np.expand_dims(clean_sounds, 1)

  noise_sounds = np.stack(noise_sounds, 0)
  noise_sounds = np.expand_dims(noise_sounds, 1)

  noisy_sounds = np.stack(noisy_sounds, 0)
  noisy_sounds = np.expand_dims(noisy_sounds, 1)
  
  return clean_sounds, noise_sounds, noisy_sounds

In [9]:
def get_cl_datas(speechs, noises1, snrs1, noises2, snrs2, length:int=16384):
  assert len(speechs) == len(noises1), 'The number of voices must be equal to the number of noises1'
  assert len(speechs) == len(noises2), 'The number of voices must be equal to the number of noises2'
  bz = len(speechs)
  clean_sounds = []
  noise_sounds1 = []
  noisy_sounds1 = []
  noise_sounds2 = []
  noisy_sounds2 = []
  targets = []
  for i in range(bz):
    speech = np.array(speechs[i])
    if len(speech)-length<0:
      speech = np.pad(speech,(0,length-len(speech)),'wrap')
    else:
      speech_start = random.randrange(0,len(speech)-length+1)
      speech = speech[speech_start:speech_start+length]
    speech = speech - np.mean(speech) 
    speech = speech / np.linalg.norm(speech,2)
    speech_std = np.std(speech)

    noise1 = np.array(noises1[i])
    if len(noise1)-length<0:
      noise1 = np.pad(noise1,(0,length-len(noise1)),'reflect')
    else:
      noise_start = random.randrange(0,len(noise1)-length+1)
      noise1 = noise1[noise_start:noise_start+length]
    noise1 = noise1 - np.mean(noise1) 
    noise1 = noise1 / np.linalg.norm(noise1,2)
    noise1 = noise1 / (10.0 ** (0.05 * snrs1[i]))
    noise1_std = np.std(noise1)

    noise2 = np.array(noises2[i])
    if len(noise2)-length<0:
      noise2 = np.pad(noise2,(0,length-len(noise2)),'reflect')
    else:
      noise_start = random.randrange(0,len(noise2)-length+1)
      noise2 = noise2[noise_start:noise_start+length]
    noise2 = noise2 - np.mean(noise2) 
    noise2 = noise2 / np.linalg.norm(noise2,2)
    noise2 = noise2 / (10.0 ** (0.05 * snrs2[i]))
    noise2_std = np.std(noise2)

    noisy1 = speech + noise1
    noisy1_std = np.std(noisy1)
    noisy1 = noisy1 / noisy1_std

    noisy2 = speech + noise2
    noisy2_std = np.std(noisy2)
    noisy2 = noisy2 / noisy2_std

    speech = speech / speech_std
    
    noise1 = noise1 / noise1_std

    noise2 = noise2 / noise2_std
    
    clean_sounds.append(speech)

    noise_sounds1.append(noise1)
    noisy_sounds1.append(noisy1)

    noise_sounds2.append(noise2)
    noisy_sounds2.append(noisy2)

  clean_sounds = np.stack(clean_sounds, 0)
  clean_sounds = np.expand_dims(clean_sounds, 1)

  noise_sounds1 = np.stack(noise_sounds1, 0)
  noise_sounds1 = np.expand_dims(noise_sounds1, 1)

  noisy_sounds1 = np.stack(noisy_sounds1, 0)
  noisy_sounds1 = np.expand_dims(noisy_sounds1, 1)

  noise_sounds2 = np.stack(noise_sounds2, 0)
  noise_sounds2 = np.expand_dims(noise_sounds2, 1)

  noisy_sounds2 = np.stack(noisy_sounds2, 0)
  noisy_sounds2 = np.expand_dims(noisy_sounds2, 1)
  
  return clean_sounds, noise_sounds1, noisy_sounds1, noise_sounds2, noisy_sounds2

In [10]:
class SI_SNR(torch.nn.Module):
  def __init__(self,eps=1e-04):
    super(SI_SNR, self).__init__()
    self.eps = torch.tensor(eps).to(device)

  def forward(self, pred, target,):
    s_target = target*(torch.sum(pred*target,(1,2),keepdim=True)/torch.maximum(torch.sum(target**2,(1,2),keepdim=True),self.eps))
    e_noise = pred-s_target
    
    sisnr = 10*torch.log10(torch.maximum(torch.sum(s_target**2,(1,2),keepdim=True),self.eps)/torch.maximum(torch.sum(e_noise**2,(1,2),keepdim=True),self.eps))
    return sisnr.mean()

In [11]:
class Block(nn.Module):
  def __init__(self, in_dim, dim, k_size, dilation=1, chunk_size=1,):
    super().__init__()
    self.conv1 = nn.Sequential(
        nn.BatchNorm1d(in_dim),
        nn.Conv1d(in_dim,dim*2,1),
        nn.GELU(),
      )
    if dilation==1:
      self.conv2 = nn.Sequential(
          nn.BatchNorm1d(dim),
          nn.Conv1d(dim,dim,k_size,padding=k_size//2,groups=dim//chunk_size,),
          nn.GELU(),
        )
    else:
      self.conv2 = nn.Sequential(
          nn.BatchNorm1d(dim),
          nn.Conv1d(dim,dim,dilation*2-1,padding=(dilation*2-1)//2,groups=dim//chunk_size),
          nn.GELU(),
          nn.Conv1d(dim,dim,k_size,padding=(k_size//2)*dilation,groups=dim//chunk_size,dilation=dilation),
          nn.GELU(),
        )
    self.conv3 = nn.Sequential(
        nn.BatchNorm1d(dim),
        nn.Conv1d(dim,in_dim,1),
        nn.GELU(),
      )

  def forward(self, x):
    z1 = self.conv1(x)
    u, v = torch.chunk(z1,2,dim=1)
    z2 = u + self.conv2(v)
    output = self.conv3(z2) + x
    return output

In [12]:
class Model(nn.Module):
  def __init__(self, patch_size = 64,
            dim = 128,
            expansion_factor = 2,
            proj_depth = 4,
            pred_depth = 2,
            k_size = 9,
            dilation = 16,
            chunk_size = 8,):
    super().__init__()
    self.cnn_enc = nn.Sequential(
          nn.Conv1d(1,dim//4,5,padding=2),
          nn.GELU(),
          nn.MaxPool1d(4,4),
          nn.BatchNorm1d(dim//4),
          
          nn.Conv1d(dim//4,dim//2,5,padding=2,groups=4),
          nn.GELU(),
          nn.MaxPool1d(4,4),
          nn.BatchNorm1d(dim//2),
          
          nn.Conv1d(dim//2,dim,5,padding=2,groups=4),
          nn.GELU(),
          nn.MaxPool1d(4,4),
          nn.BatchNorm1d(dim),
        )

    self.enc = nn.Sequential(
              Rearrange('b c (l p) -> b (p c) l', p = patch_size),
              nn.Conv1d(patch_size, dim, 3, padding=1),
            )
    
    self.project_net = nn.Sequential(
              nn.Conv1d(dim*2, dim, 1),
              *[Block(dim, dim*expansion_factor, k_size, dilation, chunk_size,) for _ in range(proj_depth)],
            )
    
    self.predict_net = nn.Sequential(
              *[Block(dim, dim*expansion_factor, k_size, dilation, chunk_size,) for _ in range(pred_depth)],
            )
    
    self.dec = nn.Sequential(
              nn.Conv1d(dim, patch_size, 3, padding=1),
              Rearrange('b (p c) l -> b c (l p)', p = patch_size),
            )

  def forward(self, x):
    z = self.project(x)
    p = self.predict(z)
    output = self.decode(p)
    return output

  def project(self, x):
    h = torch.cat([self.enc(x),self.cnn_enc(x)],dim=1)
    z = self.project_net(h)
    return z

  def predict(self, z):
    p = self.predict_net(z)
    return p

  def decode(self, p):
    output = self.dec(p)
    return output

In [13]:
def D(p, z):
  z = z.detach()
  p = p/torch.norm(p,dim=1,keepdim=True)
  z = z/torch.norm(z,dim=1,keepdim=True)

  return -(p*z).sum(dim=1).mean()

In [14]:
patch_size = 64
dim = 128
expansion_factor = 2
proj_depth = 4
pred_depth = 2
k_size = 9
dilation = 16
chunk_size = 32
model = Model(patch_size,dim,expansion_factor,proj_depth,pred_depth,k_size,dilation,chunk_size).to(device)

In [15]:
sum(p.numel() for p in model.parameters())

2666368

In [16]:
cl_epochs = 50
se_epochs = 50
rounds = 4
train_bz = 128
test_bz = 32

train_speechs = np.load("../dataset/train_speechs.npy",allow_pickle=True)
train_noises = np.load("../dataset/train_noises.npy",allow_pickle=True)
train_snr_class = [-10, -5, 0, 5, 10]

test_speechs = np.load("../dataset/test_speechs.npy",allow_pickle=True)
test_noises = np.load("../dataset/test_noises.npy",allow_pickle=True)
test_snr_class = [-7.5, -2.5, 2.5, 7.5,]

In [17]:
sisnr_fn = SI_SNR()
opt = torch.optim.SGD([
    *model.parameters(),
  ],lr=0.05,momentum=0.9,weight_decay=0.0001)

In [18]:
model.load_state_dict(torch.load("../init_model.pt"))
# torch.save(model.state_dict(), "../init_model.pt")

# checkpoint = torch.load("./state.pt")
# model.load_state_dict(checkpoint['model_state_dict'])
# opt.load_state_dict(checkpoint['opt_state_dict'])
# start_epoch = checkpoint['epoch']+1

In [19]:
def clse_train(round):

  for epoch in range(cl_epochs):
    # train
    model.train()
    train_indices = [i for i in range(len(train_speechs))]
    random.shuffle(train_indices)
    train_bnum = len(train_indices)//train_bz
    train_sisnr = 0
    train_cl_loss = 0
    for bidx in tqdm(range(train_bnum)):
      speechs = train_speechs[train_indices[bidx*train_bz:(bidx+1)*train_bz]]
      noises1 = train_noises[random.choices(range(len(train_noises)), k=train_bz)]
      snrs1 = random.choices(train_snr_class, k=train_bz)
      noises2 = train_noises[random.choices(range(len(train_noises)), k=train_bz)]
      snrs2 = random.choices(train_snr_class, k=train_bz)
      # snrs2 = snrs1

      clean_sounds, _, noisy_sounds1, _, noisy_sounds2 = get_cl_datas(speechs, noises1, snrs1, noises2, snrs2, 16384)

      clean_sounds = torch.tensor(clean_sounds).to(device)
      noisy_sounds1 = torch.tensor(noisy_sounds1).to(device)
      noisy_sounds2 = torch.tensor(noisy_sounds2).to(device)
      noisy_sounds = torch.cat([noisy_sounds1,noisy_sounds2],dim=0)
      
      opt.zero_grad()

      z = model.project(noisy_sounds)
      p = model.predict(z)
      clean_pred = model.decode(p)
      clean_pred -= torch.mean(clean_pred,dim=-1,keepdim=True)

      z1, z2 = torch.chunk(z,2,dim=0)
      p1, p2 = torch.chunk(p,2,dim=0)
      clean_pred1, clean_pred2 = torch.chunk(clean_pred,2,dim=0)

      sisnr = sisnr_fn(clean_pred1,clean_sounds)/2+sisnr_fn(clean_pred2,clean_sounds)/2
      cl_loss = D(p1,z2)/2+D(p2,z1)/2
      loss = cl_loss-sisnr*0.1
      
      loss.backward()

      opt.step()
      
      train_sisnr += sisnr.item()
      train_cl_loss += cl_loss.item()

    train_sisnr = train_sisnr/train_bnum
    train_cl_loss = train_cl_loss/train_bnum

    # eval
    model.eval()
    test_indices = [i for i in range(len(test_speechs))]
    random.shuffle(test_indices)
    test_bnum = len(test_indices)//test_bz
    test_sisnr = 0
    for _ in range(4):
      for bidx in range(test_bnum):
        speechs = test_speechs[test_indices[bidx*test_bz:(bidx+1)*test_bz]]
        noises = test_noises[random.choices(range(len(test_noises)), k=test_bz)]
        snrs = random.choices(test_snr_class, k=test_bz)
        clean_sounds, noise_sounds, noisy_sounds = get_datas(speechs, noises, snrs, 16384*2)

        clean_sounds = torch.tensor(clean_sounds).to(device)
        noise_sounds = torch.tensor(noise_sounds).to(device)
        noisy_sounds = torch.tensor(noisy_sounds).to(device)

        clean_preds = model(noisy_sounds)
        clean_preds -= torch.mean(clean_preds,dim=-1,keepdim=True)
        
        sisnr = sisnr_fn(clean_preds, clean_sounds)
        
        test_sisnr += sisnr.item()

        gc.collect()
        torch.cuda.empty_cache()

    test_sisnr = test_sisnr/(test_bnum*4)

    print("Train SISNR: {:.3f}, Test SISNR: {:.3f}, Train CL Loss: {:.3f}".format(train_sisnr, test_sisnr, train_cl_loss))

    with open('./history.csv', 'a+', newline='') as csvfile:
      writer = csv.writer(csvfile)
      # 寫入資料
      writer.writerow([train_sisnr, test_sisnr, train_cl_loss])

    torch.save({
            'epoch': epoch,
            'round': round,
            'mode': 'CLSE',
            'model_state_dict': model.state_dict(),
            'opt_state_dict': opt.state_dict(),
            'train_sisnr': train_sisnr,
            'test_sisnr': test_sisnr,
            'train_cl_loss': train_cl_loss,
            }, "./state.pt")
  

In [20]:
def se_train(round):
  for epoch in range(se_epochs):
    # train
    model.train()
    train_indices = [i for i in range(len(train_speechs))]
    random.shuffle(train_indices)
    train_bnum = len(train_indices)//train_bz
    train_sisnr = 0
    train_cl_loss = 0
    for bidx in tqdm(range(train_bnum)):
      speechs = train_speechs[train_indices[bidx*train_bz:(bidx+1)*train_bz]]
      noises1 = train_noises[random.choices(range(len(train_noises)), k=train_bz)]
      snrs1 = random.choices(train_snr_class, k=train_bz)
      noises2 = train_noises[random.choices(range(len(train_noises)), k=train_bz)]
      snrs2 = random.choices(train_snr_class, k=train_bz)
      # snrs2 = snrs1

      clean_sounds, _, noisy_sounds1, _, noisy_sounds2 = get_cl_datas(speechs, noises1, snrs1, noises2, snrs2, 16384)

      clean_sounds = torch.tensor(clean_sounds).to(device)
      noisy_sounds1 = torch.tensor(noisy_sounds1).to(device)
      noisy_sounds2 = torch.tensor(noisy_sounds2).to(device)
      noisy_sounds = torch.cat([noisy_sounds1,noisy_sounds2],dim=0)
      
      opt.zero_grad()

      z = model.project(noisy_sounds)
      p = model.predict(z)
      clean_pred = model.decode(p)
      clean_pred -= torch.mean(clean_pred,dim=-1,keepdim=True)

      z1, z2 = torch.chunk(z,2,dim=0)
      p1, p2 = torch.chunk(p,2,dim=0)
      clean_pred1, clean_pred2 = torch.chunk(clean_pred,2,dim=0)

      sisnr = sisnr_fn(clean_pred1,clean_sounds)/2+sisnr_fn(clean_pred2,clean_sounds)/2
      cl_loss = D(p1,z2)/2+D(p2,z1)/2
      loss = -sisnr
      
      loss.backward()

      opt.step()
      
      # with torch.no_grad():
      #   for (tar_p,p) in zip(target_conv_block.parameters(), conv_block.parameters()):
      #     tar_p.set_(0.9*tar_p+0.1*p)
      #   for (tar_p,p) in zip(target_encoder.parameters(), encoder.parameters()):
      #     tar_p.set_(0.9*tar_p+0.1*p)
      
      # total_loss += sound_loss.item()
      train_sisnr += sisnr.item()
      train_cl_loss += cl_loss.item()

    train_sisnr = train_sisnr/train_bnum
    train_cl_loss = train_cl_loss/train_bnum

    # eval
    model.eval()
    test_indices = [i for i in range(len(test_speechs))]
    random.shuffle(test_indices)
    test_bnum = len(test_indices)//test_bz
    test_sisnr = 0
    for _ in range(4):
      for bidx in range(test_bnum):
        speechs = test_speechs[test_indices[bidx*test_bz:(bidx+1)*test_bz]]
        noises = test_noises[random.choices(range(len(test_noises)), k=test_bz)]
        snrs = random.choices(test_snr_class, k=test_bz)
        clean_sounds, noise_sounds, noisy_sounds = get_datas(speechs, noises, snrs, 16384*2)

        clean_sounds = torch.tensor(clean_sounds).to(device)
        noise_sounds = torch.tensor(noise_sounds).to(device)
        noisy_sounds = torch.tensor(noisy_sounds).to(device)

        clean_preds = model(noisy_sounds)
        clean_preds -= torch.mean(clean_preds,dim=-1,keepdim=True)
        
        sisnr = sisnr_fn(clean_preds, clean_sounds)
        
        test_sisnr += sisnr.item()

        gc.collect()
        torch.cuda.empty_cache()

    test_sisnr = test_sisnr/(test_bnum*4)

    print("Train SISNR: {:.3f}, Test SISNR: {:.3f}, Train CL Loss: {:.3f}".format(train_sisnr, test_sisnr, train_cl_loss))

    with open('./history.csv', 'a+', newline='') as csvfile:
      writer = csv.writer(csvfile)
      # 寫入資料
      writer.writerow([train_sisnr, test_sisnr, train_cl_loss])

    torch.save({
            'epoch': epoch,
            'round': round,
            'mode': 'SE',
            'model_state_dict': model.state_dict(),
            'opt_state_dict': opt.state_dict(),
            'train_sisnr': train_sisnr,
            'test_sisnr': test_sisnr,
            'train_cl_loss': train_cl_loss,
            }, "./state.pt")

In [21]:
with open('./history.csv', 'w', newline='') as csvfile:
  # 定義欄位
  fieldnames = ['Train SISNR', 'Test SISNR', 'Train CL Loss']

  # 將 dictionary 寫入 CSV 檔
  writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

  # 寫入第一列的欄位名稱
  writer.writeheader()

for round in range(rounds):
  clse_train(round)
  se_train(round)

100%|██████████| 32/32 [00:21<00:00,  1.49it/s]


Train SISNR: 10.068, Test SISNR: 9.027, Train CL Loss: -0.633


  0%|          | 0/32 [00:00<?, ?it/s]

Train SISNR: 10.828, Test SISNR: 8.926, Train CL Loss: -0.822


  0%|          | 0/32 [00:00<?, ?it/s]

...

Train SISNR: 12.280, Test SISNR: 9.257, Train CL Loss: -0.346


  0%|          | 0/32 [00:00<?, ?it/s]

Train SISNR: 12.306, Test SISNR: 9.153, Train CL Loss: -0.360


100%|██████████| 32/32 [00:22<00:00,  1.42it/s]


Train SISNR: 12.377, Test SISNR: 9.281, Train CL Loss: -0.362
